In [2]:
import numpy as np
import pandas as pd
import os

import csv
import yaml

from rdflib import Graph, Literal, RDF, URIRef
from rdflib.namespace import Namespace

from tqdm import tqdm
from collections import defaultdict

### DRKG Import

In [3]:
drkg_file = 'data/drkg/drkg.tsv'
drkg_df = pd.read_csv(drkg_file, sep="\t", header=None)     # header fix to original code
drkg_triplets_list = drkg_df.values.tolist()

### DrugMechDB Import

In [4]:
drugmech_path = "data/drugmech/drugmechdb.yaml"

with open(drugmech_path, 'r') as fh:
        dmdb_yaml = yaml.safe_load(fh)

In [4]:
# from dmdb_data_tools_analysis import *

# all_metapath_nodes = get_metapath_node(dmdb_yaml)
# all_metapath_edges = get_metapath_edges(dmdb_yaml)
# basic_stats = defaultdict(list)

# all_metaedges = []
# all_parings = []
# all_targets = []
# unique_metaedges = []
# first_edge_type = []
# all_nodes = []

# id_to_name = {}
# id_to_label = {}

# for i, p in enumerate(dmdb_yaml):
#     _id = (p["graph"]["_id"])
#     drug_id, dis_id = path_to_tup(p)
#     paths = get_all_paths(p)
#     G = path_to_G(p)
    
#     G = add_metaedges(G)
#     G = add_meanode_pairs(G)
    
#     basic_stats['idx'].append(i) #index
#     basic_stats['id'].append(p['graph']['_id']) #DrugMechDB id
#     basic_stats['drug'].append(drug_id) #Drug id
#     basic_stats['disease'].append(dis_id)#Disease id
#     basic_stats['nodes'].append((G.nodes)) #nodes in metapath
#     basic_stats['n_nodes'].append(len(G.nodes)) # number of nodes in metapath
#     basic_stats['n_edges'].append(len(G.edges)) #number of edges in metapath
#     basic_stats['n_paths'].append(len(all_metapath_nodes[_id])) #number of paths
#     basic_stats['metapath'].append(all_metapath_nodes[_id])
#     basic_stats['metapath_with_edges'].append(all_metapath_edges[_id])

    
#     this_metaedges = [G.edges[e]['metaedge'] for e in G.edges]
    
#     all_metaedges += this_metaedges
#     unique_metaedges += list(set(this_metaedges))
    
#     all_parings += [G.edges[e]['mn_pair'] for e in G.edges]
#     all_targets += get_targets(G)
#     first_edge_type += get_target_metaedges(G)
#     all_nodes += list(G.nodes)
    
#     id_to_label = {**id_to_label, **get_id_to_type(G)}
#     id_to_name = {**id_to_name, **get_id_to_name(G)}
    
# dmdb_df = pd.DataFrame(basic_stats)

<!-- ### DruugMechDB Import -->

## Entity Dictionaries

### DRKG

In [ ]:
def insert_entry(entry, ntype, dic):
    if ntype not in dic:
        dic[ntype] = {}
    node_id = len(dic[ntype])
    if entry not in dic[ntype]:
         dic[ntype][entry] = node_id
    return dic

In [15]:
def insert_pred(entry, dic):
    # if ntype not in dic:
    #     dic[ntype] = {}
    # node_id = len(dic[ntype])
    node_id = len(dic)
    if entry not in dic.keys():
         dic[entry] = node_id
    return dic

In [16]:
def insert_entry_unique_ID(entity,ent_type,dic,total_count):
    if ent_type not in dic:
        dic[ent_type]={}
    # ent_n_id=len(dic[ent_type])
    if entity not in dic[ent_type]:
        dic[ent_type][entity]= total_count # + ent_n_id
        total_count+=1
    return dic, total_count

In [ ]:
drkg_entity_dictionary={}
e_total_count = 0

drkg_predicate_dictionary={}

drkg_triplets_id_list=[]

def find_drkg_entity_id(entity='Gene::2157'):
    etype, e_name = entity.split('::')
    return drkg_entity_dictionary[etype][e_name]

def find_drkg_entity_name(entity_id=0):
    for ent_dict in drkg_entity_dictionary.values():
        if entity_id in ent_dict.values():
            for e_name, id in ent_dict.items():
                if id == entity_id:
                    return e_name
                
def find_drkg_pred_name(pred_id=42):
    for p_name,p_id in drkg_predicate_dictionary.items():
        if p_id == pred_id:
            return p_name
   

for triple in drkg_triplets_list:
    e_head,pred,e_tail = triple

    head_type, head = e_head.split('::')
    tail_type, tail = e_tail.split('::')
    drkg_entity_dictionary,e_total_count = insert_entry_unique_ID(head,head_type,drkg_entity_dictionary,e_total_count)
    drkg_entity_dictionary,e_total_count = insert_entry_unique_ID(tail,tail_type,drkg_entity_dictionary,e_total_count)

    drkg_predicate_dictionary = insert_pred(pred,drkg_predicate_dictionary)

    drkg_triplets_id_list.append((find_drkg_entity_id(e_head),drkg_predicate_dictionary[pred],find_drkg_entity_id(e_tail)))
    

In [ ]:
tmep_set = set()

for trip in drkg_triplets_id_list:
    h,p,t = trip
    if p in tmep_set: continue
    print(trip)
    tmep_set.add(p) 
    # if h == t:
    #     print(trip)

In [ ]:
for k,v in drkg_entity_dictionary.items():
    print(len(v),k)#,v),

39220 Gene
24313 Compound
5103 Disease
4048 Atc
215 Tax
11381 Biological Process
415 Symptom
400 Anatomy
2884 Molecular Function
345 Pharmacologic Class
1391 Cellular Component
1822 Pathway
5701 Side Effect


DrugMechDB Note: There are 52 entries where drug_mesh ID exists but is not used, instead DB id was used in links and nodes

### DrugMechDB

In [23]:
dmdb_entity_dictionary = {}
e_total_count = 0

dmdb_predicate_dictionary = {}

# dmdb_triplet_dictionary = {}
dmdb_subgraph_list = []

def find_dmdb_entity_id(entity='MESH:D000068877'):
    for ent_dict in dmdb_entity_dictionary.values():
        if entity in ent_dict:
            return ent_dict[entity]

def find_dmdb_entity_name(entity_id=241):
    for ent_dict in dmdb_entity_dictionary.values():
        for e_name, id in ent_dict.items():
            if id == entity_id:
                return e_name
            
def find_dmdb_pred_id(pred='treats'):
    return dmdb_predicate_dictionary[pred]

def find_dmdb_pred_name(pred_id=4):         # TODO: separate in a utils.py (although these make more sense as a class function)
    for p_name,p_id in dmdb_predicate_dictionary.items():
        if p_id == pred_id:
            return p_name

def calculate_all_paths(triplet_set, start_id):
    map = {}
    for u,p,v in triplet_set:
        if u not in map:
            map[u]  = []
        map[u].append(v)
    path_log = {}

    def dfs(u, seen):
        if u in seen: return []                # break
        if u in path_log: return path_log[u]
        neighbors = map.get(u, ())
        if not neighbors:                      # no path left
            path_log[u] = [(u,)]
            return path_log[u]
        paths = []
        for v in neighbors:                    # exploration here
            for p in dfs(v, seen | {u}):       # seen-union-{u}
                paths.append((u,) + p)
        path_log[u] = paths
        return paths

    return [list(path) for path in dfs(start_id, set())]


for entry in dmdb_yaml:
    dbg_flag = 0
    drug_db = entry['graph']['drugbank']
    drug_mesh = entry['graph']['drug_mesh']       ## will be checking for null
    disease_mesh = entry['graph']['disease_mesh']

    is_mesh = False

    for node in entry['nodes']:     # Fill entity dictionary
        ent_id = node['id']
        ent_type = node['label']
        dmdb_entity_dictionary, e_total_count = insert_entry_unique_ID(ent_id,ent_type,dmdb_entity_dictionary,e_total_count)
        
        if drug_mesh is not None and ent_id == drug_mesh:   # if drug MESH used in links, add drug_db id separately
            dmdb_entity_dictionary, e_total_count = insert_entry_unique_ID(drug_db,ent_type,dmdb_entity_dictionary,e_total_count)
            
            is_mesh = True

    drug_id = find_dmdb_entity_id(drug_mesh) if is_mesh else find_dmdb_entity_id(drug_db)

    triplet_list = []
    entity_set = set()
    for triplet in entry['links']:  # Fill triplet dictionary
        src = triplet['source']
        dest = triplet['target']
        src_id, dest_id = find_dmdb_entity_id(src), find_dmdb_entity_id(dest)
        pred = triplet['key']
        dmdb_predicate_dictionary = insert_pred(pred,dmdb_predicate_dictionary)
        pred_id = dmdb_predicate_dictionary[pred]
        triplet_list.append((src_id, pred_id, dest_id))
        entity_set.add(src_id)
        entity_set.add(dest_id)
        # pred_set.add(pred_id)
    
    paths = calculate_all_paths(triplet_list,start_id=drug_id)
    longest_path = max(paths, key=len)
    dict_entry = {'subgraph_id': len(dmdb_subgraph_list), 
                  #'subgraph_id': len(dmdb_triplet_dictionary),
                    'drug_id': drug_id,
                    'drug_db_id': find_dmdb_entity_id(drug_db),
                    'drug_mesh_id': find_dmdb_entity_id(drug_mesh),
                    'disease_id': find_dmdb_entity_id(disease_mesh),
                    'triplets': triplet_list,
                    'entities': entity_set,
                    'paths': paths,
                    'longest_path': longest_path,
                    'longest_path_length': len(longest_path)
                }
    dmdb_subgraph_list.append(dict_entry)


In [24]:
def a(): # wow useful
    a = []

    def b(n):
        a.append(n)
        n = n-1
        if n == -1:
            return
        return b(n)    
    b(5)
    print(a)
a()

[5, 4, 3, 2, 1, 0]


In [25]:
for k,v in dmdb_entity_dictionary.items():
    print(len(v),k)#,v)

3135 Drug
788 Protein
764 Disease
694 BiologicalProcess
360 ChemicalSubstance
81 Pathway
122 GrossAnatomicalStructure
98 MolecularActivity
166 OrganismTaxon
137 GeneFamily
45 CellularComponent
274 PhenotypicFeature
37 Cell
2 MacromolecularComplex


In [29]:
for k,v in dmdb_subgraph_list[3500].items():
    print(k,v)

print('\nNumber of total paths', len(dmdb_subgraph_list[3500]['paths']))

subgraph_id 3500
drug_id 6201
drug_db_id 6202
drug_mesh_id 6201
disease_id 677
triplets [(6201, 0, 965), (965, 2, 1061), (965, 15, 2329), (1061, 24, 676), (2329, 24, 676), (676, 1, 677)]
entities {676, 1061, 965, 2329, 677, 6201}
paths [[6201, 965, 1061, 676, 677], [6201, 965, 2329, 676, 677]]
longest_path [6201, 965, 1061, 676, 677]
longest_path_length 5

Number of total paths 2


In [27]:
find_dmdb_entity_name(6201)

'MESH:C578009'

In [28]:
find_dmdb_entity_id('MESH:C578009')

6201

### Inverse Dicts for Entity Search 

In [96]:
#dmdb_inv_ent_dict = {v: k for k, v in dmdb_entity_dictionary.items()}
dmdb_inverse_ent_dict = {}
for subdict in dmdb_entity_dictionary.values():
    for k, v in subdict.items():
        dmdb_inverse_ent_dict[v]=k


#drkg_inv_ent_dict = {v: k for k, v in drkg_entity_dictionary.items()}
drkg_inverse_ent_dict = {}
for subdict in drkg_entity_dictionary.values():
    for k, v in subdict.items():
        drkg_inverse_ent_dict[v]=k

In [ ]:
dmdb_inverse_ent_dict[1000]

'5241'

## Discovering format-defying exceptions

In [ ]:
# TODO: How to handle exceptions

def print_all_type(list,feat):
    for l in list['nodes']:
        print(l[feat])

for entry in dmdb_yaml:
    if entry['nodes'][0]['label'] == 'Drug' and entry['nodes'][-1]['label'] == 'Disease':
        for idx, node in enumerate(entry['nodes']):
            if idx == 0 or idx == (len(entry['nodes'])-1): continue
            elif node['label'] == 'Drug' or node['label'] == 'Disease':
                print_all_type(entry,'label')
                print()
    else:
        print("\t", entry['nodes'][0]['label'], entry['nodes'][-1]['label'])
        print()


### Notes

        drug: Ammonium lactate
        drug_mesh: MESH:null

DrugMechDB contains both  MESH:null and null for drug_mesh, and only one null for drugbank section.

        drug: tazanolast
        drug_mesh: MESH:C106301
        drugbank: null

And only two instances of double entries in drug_mesh:

        drug: acrisorcin
        drug_mesh: MESH:D000585, MESH:D006604
        drugbank: DB:DB11254

        drug: carfilzomib
        drug_mesh: MESH:C524865, MESH:C519125
        drugbank: DB:DB08889

In [22]:
print(len(dmdb_entity_dictionary['Drug']))
print(len(drkg_entity_dictionary['Compound']))
print()
print(len(dmdb_entity_dictionary['Disease']))
print(len(drkg_entity_dictionary['Disease']))

3135
24313

764
5103


In [23]:
def print_sample(dic, key):
    for k,v in dic[key].items():
        print(k)
        break

print_sample(dmdb_entity_dictionary, 'Drug')
print_sample(drkg_entity_dictionary, 'Compound')
print()
print_sample(dmdb_entity_dictionary, 'Disease')
print_sample(drkg_entity_dictionary, 'Disease')

MESH:D000068877
DB02573

MESH:D015464
SARS-CoV2 E


In [25]:
# 1 DrugBank ID is null

cnt = 0
for entry in dmdb_yaml:
    elem = entry['graph']['drugbank']
    if elem is None: 
        print("null")
        continue
    if "DB" in elem:
        cnt += 1
cnt

null


4845

In DRKG;

Compound::DB00898
Compound::MESH:C106876      Disease::MESH:C537014
Compound::MESH:D000588      Disease::MESH:D013274

Compound::MESH:C554292|MESH:D013755


In DMDB;

drugbank: DB:DB09310
drug_mesh: MESH:D000068877  disease_mesh: MESH:C567691

drug_mesh: MESH:C524865, MESH:C519125

# Drugs, Diseases, & Commons

## Drugs

In [30]:
def insert_entry_eid(entity,ent_type,eid,dic):
    if ent_type not in dic:
        dic[ent_type]={}
    ent_n_id=eid
    if entity not in dic[ent_type]:
        dic[ent_type][entity]=ent_n_id
    return dic

In [31]:
dmdb_drug_dict = {}

for drug, eid in dmdb_entity_dictionary['Drug'].items():
    if drug != None:
        identifier, idx = drug.split(":")
    insert_entry_eid(idx, identifier, eid, dmdb_drug_dict)

In [ ]:
dmdb_entity_dictionary['Drug'][None]    # TODO: Not okay

241

In [ ]:
### Quick detour - somehow None is in entities dictionary but not in any subgraphs
count = 0
for subgraph in dmdb_subgraph_list:
    if 241 in subgraph['entities']:
        count += 1
count

0

In [61]:
for k,v in dmdb_drug_dict.items():
    print(len(v),k)#,v)

1556 MESH
1577 DB
1 CHEBI


In [34]:
drkg_drug_dict = {}

for drug,eid in drkg_entity_dictionary['Compound'].items():
    if len(drug.split(":")) == 1:    # DB and CHEMBL
        if 'DB' in drug:
            insert_entry_eid(drug, 'DB', eid, drkg_drug_dict)
        elif 'CHEMBL' in drug: 
            insert_entry_eid(drug, 'CHEMBL', eid, drkg_drug_dict)
        else:
            print("DEBUG Outlier with no split:", drug)  
    else:
        if "|" in drug:       # Exceptions  --  MESH:C002480|MESH:, MESH:C011440|MESH:C046229,...
            continue
            # TODO: figure how to handle them appropriately
        else:
            identifier, idx = drug.split(":")
            insert_entry_eid(idx, identifier, eid, drkg_drug_dict)

In [35]:
for k,v in drkg_drug_dict.items():
    print(len(v),k)#,v)

10551 DB
228 molport
731 brenda
51 gtopdb
144 bindingdb
98 pubchem
63 zinc
25 chebi
23 rhea
18 drugcentral
2 fdasrs
4 nmrshiftdb2
12 hmdb
3665 CHEMBL
12 Bioarxivdrug
1 pharmgkb
7605 MESH
1063 CHEBI


In [38]:
# common_drug_set = set()
common_drug_dict = defaultdict(tuple)

counter = [0,0,0]
for mesh_drug,dmdb_id in dmdb_drug_dict['MESH'].items():
    if mesh_drug in drkg_drug_dict['MESH'].keys():
        # common_drug_set.add('MESH:'+ mesh_dis)       # Re-adding identifier 'MESH:'
        common_drug_dict['MESH:'+ mesh_drug] = (dmdb_id,drkg_drug_dict['MESH'][mesh_drug])
        counter[0]+=1
for db_drug,dmdb_id in dmdb_drug_dict['DB'].items():
    if db_drug in drkg_drug_dict['DB'].keys():
        # common_drug_set.add('DB:'+ mesh_dis)       # Re-adding identifier 'DB:'
        common_drug_dict['DB:'+ db_drug] = (dmdb_id,drkg_drug_dict['DB'][db_drug])
        counter[1]+=1
for chebi_drug,dmdb_id in dmdb_drug_dict['CHEBI'].items():
    if chebi_drug in drkg_drug_dict['CHEBI'].keys():
        # common_drug_set.add('CHEBI:'+ mesh_dis)       # Re-adding identifier 'CHEBI:'
        common_drug_dict['CHEBI:'+ chebi_drug] = (dmdb_id,drkg_drug_dict['CHEBI'][chebi_drug])
        counter[2]+=1

print("Number of common drugs [MESH/DB/CHEBI]:", len(common_drug_dict), counter)

Number of common drugs [MESH/DB/CHEBI]: 1778 [214, 1564, 0]


In [42]:
for k,v in common_drug_dict.items():
    print('Common drug name:', k)
    print(f'DrugMech entity dict ID: {v[0]}, DRKG entity dict ID: {v[1]}')
    break

Common drug name: MESH:D000658
DrugMech entity dict ID: 60, DRKG entity dict ID: 38721


In [ ]:
dmdb_to_drkg_drug_id_conv = {}      # TODO: messy

for pair in common_drug_dict.values():
    dmdb_to_drkg_drug_id_conv[pair[0]] = pair[1]

## Diseases

In [43]:
dmdb_disease_dict = {}

for disease, eid in dmdb_entity_dictionary['Disease'].items():
    identifier, idx = disease.split(":")
    insert_entry_eid(idx, identifier, eid, dmdb_disease_dict)

In [44]:
for k,v in dmdb_disease_dict.items():
    print(len(v),k,v)

758 MESH {'D015464': 3, 'D034721': 7, 'D010146': 15, 'D005334': 19, 'D013927': 25, 'D004405': 35, 'D007634': 41, 'D003586': 47, 'D018805': 53, 'D000073605': 59, 'D014069': 65, 'D011023': 71, 'D016920': 74, 'D013717': 77, 'D014552': 81, 'D006973': 87, 'D009181': 93, 'D012223': 98, 'D011704': 102, 'D013345': 110, 'D006996': 114, 'D001991': 119, 'D012559': 126, 'D017449': 130, 'D015658': 135, 'D008595': 141, 'D003233': 151, 'D016649': 156, 'D006069': 161, 'D000881': 166, 'D011655': 171, 'D008060': 176, 'D007710': 180, 'D065631': 183, 'D015508': 188, 'D004827': 193, 'D003865': 202, 'D000749': 208, 'D015523': 213, 'D000505': 225, 'D007172': 231, 'D018410': 236, 'D003424': 239, 'D012148': 250, 'D010538': 254, 'D012507': 257, 'D012871': 269, 'D005879': 274, 'D000856': 276, 'D009103': 284, 'D006943': 288, 'D006976': 297, 'D005764': 302, 'D059268': 307, 'D004414': 308, 'D008088': 313, 'D010392': 316, 'D013610': 326, 'D009101': 335, 'D013964': 343, 'D005705': 351, 'D004832': 363, 'D011471': 368,

In [45]:
drkg_disease_dict = {}

for disease, eid in drkg_entity_dictionary['Disease'].items():
    if len(disease.split(":")) != 2:
        insert_entry_eid(disease, 'other', eid, drkg_disease_dict)
    else:
        identifier, idx = disease.split(":")
        insert_entry_eid(idx, identifier, eid, drkg_disease_dict)

In [46]:
for k,v in drkg_disease_dict.items():
    print(len(v),k,v)

27 other {'SARS-CoV2 E': 9079, 'SARS-CoV2 M': 9085, 'SARS-CoV2 N': 9110, 'SARS-CoV2 Spike': 9124, 'SARS-CoV2 nsp1': 9127, 'SARS-CoV2 nsp10': 9132, 'SARS-CoV2 nsp11': 9137, 'SARS-CoV2 nsp12': 9139, 'SARS-CoV2 nsp13': 9159, 'SARS-CoV2 nsp14': 9197, 'SARS-CoV2 nsp15': 9198, 'SARS-CoV2 nsp2': 9201, 'SARS-CoV2 nsp4': 9208, 'SARS-CoV2 nsp5': 9216, 'SARS-CoV2 nsp5_C145A': 9217, 'SARS-CoV2 nsp6': 9219, 'SARS-CoV2 nsp7': 9222, 'SARS-CoV2 nsp8': 9247, 'SARS-CoV2 nsp9': 9270, 'SARS-CoV2 orf10': 9286, 'SARS-CoV2 orf3a': 9295, 'SARS-CoV2 orf3b': 9303, 'SARS-CoV2 orf6': 9305, 'SARS-CoV2 orf7a': 9309, 'SARS-CoV2 orf8': 9312, 'SARS-CoV2 orf9b': 9352, 'SARS-CoV2 orf9c': 9363}
4871 MESH {'D001351': 9388, 'D045473': 9394, 'D065207': 9425, 'D028941': 9430, 'D058957': 9431, 'D006517': 9433, 'D003550': 25072, 'C063419': 25073, 'D013167': 25074, 'D006086': 25075, 'D017497': 25076, 'D001171': 25077, 'D011565': 25078, 'D015535': 25079, 'D017511': 25080, 'D001172': 25081, 'D013262': 25082, 'D054058': 25083, 'D0

In [48]:
# MESH Only
# common_disease_set = set()
common_disease_dict = defaultdict(tuple)

for mesh_dis,dmdb_id in dmdb_disease_dict['MESH'].items():
    if mesh_dis in drkg_disease_dict['MESH'].keys():
        # common_disease_set.add('MESH:'+ mesh_dis)       # Re-adding identifier 'MESH:'
        common_disease_dict['MESH:'+ mesh_dis] = (dmdb_id,drkg_disease_dict['MESH'][mesh_dis])

print("Number of common MESH diseases:", len(common_disease_dict))

Number of common MESH diseases: 714


In [ ]:
for k,v in common_disease_dict.items():
    print('Common disease name:', k)
    print(f'DrugMech entity dict ID: {v[0]}, DRKG entity dict ID: {v[1]}')
    break

Common disease name: MESH:D015464
DrugMech entity dict ID: 3, DRKG entity dict ID: 25687


In [63]:
dmdb_to_drkg_disease_id_conv = {}      # TODO: messy

for pair in common_disease_dict.values():
    dmdb_to_drkg_disease_id_conv[pair[0]] = pair[1]

# Common Pairs

TODO : Include entity dictionary IDs for keeping track, use it in paths. 

## Common entities in (drug-disease) pairs

In [51]:
common_pairs_dict = defaultdict()

for entry in dmdb_yaml:
    graph_dict = entry['graph']
    db_id = graph_dict['drugbank']
    mesh_id = graph_dict['drug_mesh']
    dis_id = graph_dict['disease_mesh']

    if db_id in common_drug_dict and db_id != None:     # Drugbank ID
        if dis_id in common_disease_dict:
            common_pairs_dict[(db_id,dis_id)] = 'DB' # = ((dmdb_drug_dict['DB'][db_id],drkg_drug_dict['DB'][db_id]),(dmdb_disease_dict['MESH'][dis_id],drkg_disease_dict['MESH'][dis_id]))
    elif mesh_id in common_drug_dict and mesh_id != None and mesh_id != 'MESH:null':    # Drug Mesh ID  - no duplicates (Drugbank ID has priority)
        if dis_id in common_disease_dict:
            common_pairs_dict[(mesh_id,dis_id)] = 'MESH'

print("Number of common drug-disease pairs:", len(common_pairs_dict),'/',len(dmdb_yaml))

Number of common drug-disease pairs: 4484 / 4846


#### Testing

In [53]:
# Testing reverse pair search
for i,common_pair in enumerate(common_pairs_dict.keys()):
    print(common_pair)
    common_drug, common_disease = common_pair
    d_type, drug = common_drug.split(':')
    dis_type, disease = common_disease.split(':')
    print("(DRUG) DMDB Entity index, DRKG Entity index:", dmdb_drug_dict[d_type][drug], drkg_drug_dict[d_type][drug])
    print("(DISEASE) DMDB Entity index, DRKG Entity index:", dmdb_disease_dict[dis_type][disease], drkg_disease_dict[dis_type][disease])
    print()
    
    # if i == 10:
    break

('DB:DB00619', 'MESH:D015464')
(DRUG) DMDB Entity index, DRKG Entity index: 1 8538
(DISEASE) DMDB Entity index, DRKG Entity index: 3 25687



In [55]:
for i,common_pair in enumerate(common_pairs_dict.keys()):
    common_drug, common_disease = common_pair
    if "DB" in common_drug:
        common_drug = common_drug.split(":")[1]
    print(f'("Compound::{common_drug}", "Disease::{common_disease}"),')
    break


("Compound::DB00619", "Disease::MESH:D015464"),


## Common Pairs in triplets

In [70]:
common_pairs_set = set()
converted_pairs_set = set()

for subgraph in dmdb_subgraph_list:
    dmdb_drug_db_id, dmdb_drug_mesh_id = subgraph['drug_db_id'], subgraph['drug_mesh_id']
    dmdb_disease_id = subgraph['disease_id']

    # Translate to drkg id if available
    if dmdb_drug_db_id in dmdb_to_drkg_drug_id_conv and dmdb_disease_id in dmdb_to_drkg_disease_id_conv:
        drug_drkg_conv_id = dmdb_to_drkg_drug_id_conv[dmdb_drug_db_id]
        dis_drkg_conv_id = dmdb_to_drkg_disease_id_conv[dmdb_disease_id]
    elif dmdb_drug_mesh_id in dmdb_to_drkg_drug_id_conv and dmdb_disease_id in dmdb_to_drkg_disease_id_conv:
        drkg_conv_id = dmdb_to_drkg_drug_id_conv[dmdb_drug_mesh_id]
        dis_drkg_conv_id = dmdb_to_drkg_disease_id_conv[dmdb_disease_id]
    else:
        # print('Pair does not exist in DRKG.')
        continue

    converted_pairs_set.add((drug_drkg_conv_id, dis_drkg_conv_id))

# Check for common pairs
for h, r, t in drkg_triplets_id_list:
    if (h, t) in converted_pairs_set:
        common_pairs_set.add((h, r, t))

In [69]:
print("Number of common pairs between DrugMechDB and DRKG", len(common_pairs_set))

Number of common pairs between DrugMechDB and DRKG 2899


# Pathing strategy for DrugMechDB

Determine the data structure formatting to store the subgraph structure. Use integer IDs in entity dictionaries and focus on connections.